In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import RobustScaler

import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots

files_path = 'files/'
output_path = 'output/'

In [ ]:
data = pd.read_csv(files_path + 'timeseries_data.csv', parse_dates=['datetime'])

# Gaps description

In [ ]:
data.set_index('datetime', inplace=True)
# Resample hourly by station and variable, filling gaps with NaN
data = (
    data
    .groupby(["variable", "station"])
    .resample("h")
    .asfreq()['value']   
    .reset_index()
)


In [ ]:
data = data.pivot_table(index=['variable', 'datetime'], columns='station', values='value', aggfunc='first', dropna=False).reset_index()

# Select time window

In [ ]:
# Filter data from 2020-01-01 onwards
date = '2020-04-01T00:00:00+00:00'
data.query(f'datetime >= "{date}"', inplace=True)

stations = [col for col in data.columns if col not in ['variable', 'datetime']]
variables = data['variable'].unique()
default_columns = ['datetime','variable']

## Gap distribution

### Nans table
Create another table but replace nan values depending on the number of consecutive
nans in this way:

Consecutive nans | Replacement     | Description
-----------------|-----------------|----------------
x = 0               |   0   | No nan value
x = 1               |   1   | Missing value for a single hour
1 > x <= 3          |   2   | Missing values for less than 3 hours
3 > x <= 12         |   3   | Missing values for more than 3 hours but less or equal to 12
12 > x <= 24        |   4   | Missing values for more than 12 hours but less or equal to 24
24 > x <= 72        |   5   | Missing values for more than 1 day but less or equal to 3 days
72 > x <= 168       |   6   | Missing values for more than 3 days but less or equal to 7 days
168 > x <= 720      |   7   | Missing values for more than 7 days but less or equal to 30 days
720 > x <= 2,160    |   8   | Missing values for more than 30 days but less or equal to 90 days
2,160 > x <= 4,320  |   9   | Missing values for more than 90 days but less or equal to 180 days
4,320 > x           |  10   | Missing values for more than 180 days

In [ ]:
# Define mapping for gap categories
map_gap = {'0': 'No gap',
           '1': '1h',
           '2': '1h > gap ≤ 3h',
           '3': '3h > gap ≤ 12h',
           '4': '12h > gap ≤ 24h',
           '5': '24h > gap ≤ 3d',
           '6': '3d > gap ≤ 7d',
           '7': '7d > gap ≤ 30d',
           '8': '30d > gap ≤ 90d',
           '9': '90d > gap ≤ 180d',
           '10': '180d > gap'
           }

In [ ]:
# Create a copy of the dataframe
df_consecutive_nans = data.copy()

# Group by the 'variable' column and sort by 'datetime'
df_consecutive_nans.sort_values(by=['datetime','variable'], inplace=True)
grouped = df_consecutive_nans.groupby('variable')


# Iterate over each group to calculate consecutive NaNs
for name, group in grouped:
    for col in group.columns:
        if col not in ['variable', 'datetime']:
            # Identify NaN values
            is_nan = group[col].isna()
            
            # Group consecutive NaNs and calculate their count
            nan_group = (is_nan != is_nan.shift()).cumsum()
            df_consecutive_nans.loc[group.index, col] = is_nan.groupby(nan_group).transform('sum')
            
            # Set non-NaN values to 0
            df_consecutive_nans.loc[group.index[~is_nan], col] = 0

# Convert all columns except 'year' and 'month' to integers
columns_to_convert = [col for col in df_consecutive_nans.columns if col not in ['variable', 'datetime',]]
df_consecutive_nans[columns_to_convert] = df_consecutive_nans[columns_to_convert].astype(int)

df_temp_gaps = df_consecutive_nans.copy()

# Define the bins and corresponding labels
bins = [0, 1, 3, 12, 24, 24*3, 24*7, 24*30, 24*30*3, 24*30*6, np.inf]
labels = np.arange(len(bins) - 1)

for column in df_temp_gaps.columns:
    if column not in ['variable', 'datetime']:
        # Replace 0 with NaN for the purpose of binning
        df_temp_gaps[column] = np.digitize(df_temp_gaps[column], bins, right=True) 

In [ ]:
bar_mode = 'stack' # 'stack' or 'group'
subplot_pan_columns = 4 # Needed if barmode is 'group'

family_font = "Times New Roman"

In [ ]:
dataframe = df_temp_gaps.copy()

# Count the number of elements equal to 0 for each column
zero_counts = (dataframe == 0).sum()

# Sort specified_order and remaining_columns by the number of zeros in descending order
specified_order = sorted(stations, key=lambda col: zero_counts[col], reverse=True)

dataframe = dataframe[default_columns + specified_order]
# Group by 'variable' and calculate the percentage of each value in each station
variable_groups = dataframe.groupby('variable')

# Create a dictionary to store the percentage data for each variable
percentage_data = {}

for variable, group in variable_groups:
    
    group.sort_values(by=['datetime'], inplace=True)
    # For each column identify with nan all values before the first 0
    for col in group.columns:
        temp = (group[col].copy()==0).cumsum() > 0
        group[col] = group[col].where(temp, np.nan)
    
    
    # Calculate the percentage for each station
    percentage_data[variable] = group.drop(columns=default_columns).apply(
        lambda x: x.value_counts(normalize=True) * 100
    ).fillna(0)
    
all_values = sorted(set(val for d in percentage_data.values() for val in d.index))
color_scale = px.colors.qualitative.Bold  # You can also use px.colors.sequential.OrRd, etc.

# If there are more values than colors in the scheme, they repeat
color_map = {val: color_scale[i % len(color_scale)] for i, val in enumerate(all_values)}

fig = make_subplots(
    rows=len(variables), 
    cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.02, 
    shared_yaxes=True,
)

# Adjust the position of subplot titles
for i, annotation in enumerate(fig['layout']['annotations']):
    annotation['y'] -= 0.005

# Iterate over variables and add traces to the subplot
for i, variable in enumerate(variables, start=1):
    if variable in percentage_data:
        data_var = percentage_data[variable].reset_index().melt(id_vars='index', var_name='Station', value_name='Percentage')
        data_var.rename(columns={'index': 'Value'}, inplace=True)
        
        for value in data_var['Value'].unique():
            filtered_data = data_var[data_var['Value'] == value]
            legend_name = map_gap.get(str(int(value)), 'Nan')

            # Add bar trace for each value, reference to the same legend group
            fig.add_trace(
                go.Bar(
                    x=filtered_data['Station'],
                    y=filtered_data['Percentage'],
                    name=legend_name,
                    legendgroup=legend_name,
                    showlegend=(i == 1),              
                    marker_color=color_map[value],
                    textposition='outside'
                ),
                row=i, col=1
            )

# Update layout for all subplots
fig.update_yaxes(
    side='left',
    range=[0, 112],
    ticksuffix='%',
    tickvals=[0, 20, 40, 60, 80, 100],
    showgrid=True,
)

# Set y-axis titles for each subplot (variable)
for i, variable in enumerate(variables, start=1):
    fig.update_yaxes(
        title_text=variable, 
        title_font=dict(size=18, family=family_font),
        tickfont=dict(size=14, family=family_font),
        row=i, col=1
    )
    
# Update axis font
fig.update_xaxes(tickfont=dict(size=14, family=family_font))

# Update layout to show legend on the top of the plot
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family=family_font)
    )
)

# Update overall figure layout
fig.update_layout(
    height=150 * len(variables),  # Adjust height based on the number of variables
    width=1000,
    barmode=bar_mode
)



fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='gray', griddash='solid')
fig.update_xaxes(showgrid=False)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

if bar_mode == 'group':
    fig.update_xaxes(
        range=[-0.5, subplot_pan_columns - 0.5],
    )

fig.show()

fig.write_html(f"output/gaps_analysis_{bar_mode}.html")


# Spatial Gap visualization by Variable

In [ ]:
df_spatial_gaps = data.melt(id_vars=['datetime', 'variable'])
df_spatial_gaps = df_spatial_gaps.groupby(['datetime','variable']).agg({'value':'count'}).reset_index()
df_spatial_gaps = df_spatial_gaps.pivot_table(values='value', index = 'datetime', columns='variable')

In [ ]:
fig = go.Figure()

# Add a line for each variable
for variable in df_spatial_gaps.columns:
    fig.add_trace(go.Scatter(
        x=df_spatial_gaps.index,
        y=df_spatial_gaps[variable],
        mode='lines',
        name=variable,
    ))

# Update general layout 
fig.update_layout(
    title=f'Network Coverage',
    title_font=dict(size=24, family=family_font),
    xaxis_title='Date',
    yaxis_title=f'Number of working stations',
    height=400,
    width=1300
)

# Update legend 
fig.update_layout(
    legend=dict(
        font=dict(size=18, family=family_font)
    )
)

# # --- Timestamps each X hours
# # Add x axis labels just for every 24 hours (change as needed)
# hours = 24
# tickvals=df_spatial_gaps.index[::hours]
# ticktext=df_spatial_gaps.index[::hours].strftime('%Y-%m-%d')

# --- Timmestamps each X day
days = ['1', '15']
tickindex=df_spatial_gaps.index.day.astype(str).isin(['1', '15'])\
    & (df_spatial_gaps.index.hour == 0)
tickvals=df_spatial_gaps.index[tickindex]
ticktext=df_spatial_gaps.index[tickindex].strftime('%Y-%m-%d')

fig.update_xaxes(tickvals= tickvals, ticktext= ticktext)


# Set the range for the x-axis default zoom
start = start=df_spatial_gaps.index.min()
end = start + pd.Timedelta(days=31)
fig.update_layout(xaxis_range=[start, end])

# Add monthly vertical lines and month names
months_ini = pd.date_range(start=df_spatial_gaps.index.min(), end=df_spatial_gaps.index.max(), freq="MS")  # 1st of each month

for ini, middle in zip(months_ini, months_ini + pd.Timedelta(days=15)):
    # Vertical line end of month
    fig.add_vline(x=ini, line_width=2, line_dash="dashdot", line_color="red")
    
    # Month name
    fig.add_annotation(
        x=middle, y=25, 
        text=middle.strftime('%B').capitalize(),
        showarrow=False,
        font=dict(size=20, color="gray", family="Times New Roman"), xanchor="center",
    )

# Add dashed grid lines with alpha 0.5
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='gray', griddash='dash')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='gray', griddash='dash')

# Update axis labels
fig.update_xaxes(title_font=dict(size=20, family=family_font),
                 tickfont=dict(size=16, family=family_font),
                 tickangle=30
                 )
fig.update_yaxes(title_font=dict(size=20, family=family_font),
                 tickfont=dict(size=16, family=family_font)
                 )


# fig.update_layout(template='plotly_dark')
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')




fig.show()
fig.write_html(f"output/Coverage2.html")

## Visualize classic time series

In [ ]:
# Select variable to analyze
variable = 'o3'

ts_var = data.query(f"variable == '{variable}'").drop(columns = ['variable'])

for col in ts_var.columns:
    # replace string 'nr' with np.nan
    ts_var[col] = ts_var[col].replace({'nr': np.nan})
    if col != 'datetime':
        ts_var[col] = pd.to_numeric(ts_var[col])
    if ts_var[col].isna().all():
        ts_var.drop(columns=[col], inplace=True)
        

ts_var.set_index('datetime', inplace=True)
ts_var.sort_index(inplace=True)

total_nans_by_column = ts_var.isna().sum()
columns_by_nans = total_nans_by_column.sort_values(ascending=True).index

ts_var = ts_var[columns_by_nans]


In [ ]:
fig = go.Figure()

# Add a line for each station
for station in ts_var.columns:
    fig.add_trace(go.Scatter(
        x=ts_var.index,
        y=ts_var[station],
        mode='lines',
        name=station
    ))

# Update general layout
fig.update_layout(
    title=f'{variable.upper()} Time series',
    title_font=dict(size=24, family=family_font),
    xaxis_title='Date',
    yaxis_title=f'{variable.upper()} Value',
    height=800,
    width=1600
)

# --- Timestamps each X hours
# Add x axis labels just for every 24 hours (change as needed)
hours = 24
tickvals=df_spatial_gaps.index[::hours]
ticktext=df_spatial_gaps.index[::hours].strftime('%Y-%m-%d')

# # --- Timmestamps each X day
# days = ['1', '15']
# tickindex=df_spatial_gaps.index.day.astype(str).isin(['1', '15'])\
#     & (df_spatial_gaps.index.hour == 0)
# tickvals=df_spatial_gaps.index[tickindex]
# ticktext=df_spatial_gaps.index[tickindex].strftime('%Y-%m-%d')

fig.update_xaxes(tickvals= tickvals, ticktext= ticktext)

# Set the range for the x-axis default zoom
start = start=df_spatial_gaps.index.min()
end = start + pd.Timedelta(days=15)
fig.update_layout(xaxis_range=[start, end])

# Add monthly vertical lines and month names
months_ini = pd.date_range(start=df_spatial_gaps.index.min(), end=df_spatial_gaps.index.max(), freq="MS")  # 1st of each month

for ini, middle in zip(months_ini, months_ini + pd.Timedelta(days=15)):
    # Vertical line end of month
    fig.add_vline(x=ini, line_width=2, line_dash="dashdot", line_color="red")
    
    # Month name
    fig.add_annotation(
        x=middle, y=200, 
        text=middle.strftime('%B').capitalize(),
        showarrow=False,
        font=dict(size=20, color="gray", family="Times New Roman"), xanchor="center",
    )

# Add dashed grid lines with alpha 0.5
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='gray', griddash='dash')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='gray', griddash='dash')

# Update axis labels
fig.update_xaxes(title_font=dict(size=20, family=family_font),
                 tickfont=dict(size=16, family=family_font),
                 tickangle=30
                 )
fig.update_yaxes(title_font=dict(size=20, family=family_font),
                 tickfont=dict(size=16, family=family_font)
                 )

# Update legend 
fig.update_layout(
    legend=dict(
        font=dict(size=16, family=family_font)
    )
)

# fig.update_layout(template='plotly_dark')
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
fig.show()
fig.write_html(f"output/timeseries_{variable}.html")